In [1]:
using Revise
using Test

In [2]:
includet("tvp-var.jl")

In [3]:
coef = generate_coeffs([0.03 0.05; 0.2 0.4], [0.8 0.8; 0.8 0.8], [0.001 0.001; 0.001 0.001], [0.01 0.02; 0.01 0.02], 8)



8×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.0449606  0.210534
 0.047042   0.203877
 0.0463575  0.212497
 0.051806   0.211082
 0.0521946  0.213788
 0.0518883  0.212587
 0.0586757  0.211801
 0.0570373  0.207326

[:, :, 2] =
 0.0628069  0.415029
 0.0630053  0.402155
 0.0680509  0.389482
 0.0675822  0.430523
 0.0517099  0.418194
 0.0548726  0.428162
 0.0431382  0.426869
 0.0345367  0.455174

In [4]:
round.(coef[8, :, :], digits=3)

2×2 Matrix{Float64}:
 0.057  0.035
 0.207  0.455

In [5]:
coef = generate_coeffs([0.03 0.05], [0.6 0.6], [0.001 0.001], [0.01 0.02], 8)

round.(coef[2, :, :], digits=3)

1×2 Matrix{Float64}:
 0.035  0.01

In [6]:
@test(residuals([0 0], [0.52 -0.35], [0.3 0.25]) == [-0.156 0.0875])


Error During Test at c:\Users\matsz\programowanie\Optymalizacja_portfela\julia_msp\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:1
  Test threw exception
  Expression: residuals([0 0], [0.52 -0.35], [0.3 0.25]) == [-0.156 0.0875]
  DimensionMismatch: A has dimensions (1,2) but B has dimensions (1,2)
  Stacktrace:
   [1] gemm_wrapper!(C::Matrix{Float64}, tA::Char, tB::Char, A::Matrix{Float64}, B::Matrix{Float64}, _add::LinearAlgebra.MulAddMul{true, true, Bool, Bool})
     @ LinearAlgebra C:\Users\matsz\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:577
   [2] generic_matmatmul!
     @ C:\Users\matsz\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:352 [inlined]
   [3] mul!
     @ C:\Users\matsz\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:263 [inlined]
   [4] mul!
     @ C:\Users\matsz\.julia\juliaup\julia-1.10.2+0

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [7]:
k_gain([0.0 0.0; 0.0 0.0], [0.45 -0.36; 0.45 -0.36], diagm([0.09, 0.12]))

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [8]:
Β = generate_coeffs([0.03 0.05; 0.2 0.4], [0.8 0.8; 0.8 0.8], [0.001 0.001; 0.001 0.001], [0.01 0.02; 0.01 0.02], 10)

10×2×2 Array{Float64, 3}:
[:, :, 1] =
  0.0197017   0.218535
  0.0169635   0.217458
  0.0336107   0.21727
  0.0452607   0.22559
  0.0358599   0.217236
  0.0285633   0.218658
  0.00477769  0.232028
 -0.0103034   0.231827
 -0.0101963   0.232285
 -0.0163757   0.231108

[:, :, 2] =
 0.0363988  0.391332
 0.0365803  0.380712
 0.0378507  0.377771
 0.036882   0.368546
 0.0365219  0.351921
 0.0368815  0.332802
 0.0506529  0.359965
 0.0492017  0.287499
 0.0762839  0.270353
 0.0758084  0.281279

In [9]:
round.(rand(Uniform(-1,1),2), digits=2)

2-element Vector{Float64}:
 -0.48
  0.32

In [10]:
Β[1,:,:]

2×2 Matrix{Float64}:
 0.0197017  0.0363988
 0.218535   0.391332

In [11]:
0.029 * -0.48 + 0.04 * -0.48

-0.03312

In [12]:
Β[1,:,:] * X[:,1]

UndefVarError: UndefVarError: `X` not defined

In [139]:
X1 = [.9 .75]
Β0 = [0.9 0.6; 0.3 0.5 ]

params = TTVAR_Result([.9 .9; .9 .9], 
    [.001 .001; .001 .001], 
    [.015 .015; .015 .015],
    diagm([.1, .1])
    )

round.(sample(X1, Β0, params, 8)', digits=2)

9×2 Matrix{Float64}:
 0.9    0.75
 0.6    0.55
 0.4    0.78
 0.24   1.0
 0.26   0.81
 0.49   0.61
 0.56   0.01
 0.4   -0.38
 0.05  -0.04

In [15]:
#init params
B0 = [0 0; 0 0]
P0 = [0 0; 0 0]
ν = [.011 .007; .009 0.021]
Y = [.035 .04]
X = [.03 .045; .03 0.45]
Σ = diagm([.01, .015])

#predict 
Β1 = B0
P1 = P0 +  ν


F = X * P1 * X' + Σ  #state transision
K = P1 * X' * inv(F) # kalman gain

res = Y .- Β1 * X #residuals

Β_f = B0 + K * res #Beta measurement
P_f = P1 - K * X * P1 #covariance measurmenet

2×2 Matrix{Float64}:
 0.0101843   0.00522623
 0.00675747  0.0160974

In [152]:
#init params
B0 = [0 0; 0 0]
P0 = [0 0; 0 0]
ν = [.011 .007; .009 0.021]
Y = [.035 .04]
X = [.03 .045; .03 0.45]
Σ = diagm([.01, .015])

kalman_step(B0, P0, Y, X, Σ, ν)

([0.008092924060680146 0.009249056069348739; 0.020620110247199317 0.023565840282513503], [0.01018431704856401 0.005226226564953576; 0.0067574702480278775 0.016097352342686905])

In [174]:
X = [0.9 0.75;
0.6 0.55;
0.4 0.78;
0.24 1.0;
0.26 0.81;
0.49 0.61;
0.56 0.01;
0.4 -0.38]

reshape(repeat(X,1,2), (8,2,2))

8×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.9    0.75
 0.6    0.55
 0.4    0.78
 0.24   1.0
 0.26   0.81
 0.49   0.61
 0.56   0.01
 0.4   -0.38

[:, :, 2] =
 0.9    0.75
 0.6    0.55
 0.4    0.78
 0.24   1.0
 0.26   0.81
 0.49   0.61
 0.56   0.01
 0.4   -0.38

In [184]:
X = [0.9 0.75;
0.6 0.55;
0.4 0.78;
0.24 1.0;
0.26 0.81;
0.49 0.61;
0.56 0.01;
0.4 -0.38]

X = reshape(repeat(X,1,2), (8,2,2))

Y = [0.6 0.55;
0.4 0.78;
0.24 1.0;
0.26 0.81;
0.49 0.61;
0.56 0.01;
0.4 -0.38;
0.05 -0.04]

B0 = [0 0; 0 0]
P0 = [0 0; 0 0]
Σ = diagm([.01, .015])
ν = [.015 .025; .015 .035]
res = kalmanFilter(X, Y, B0, P0, Σ, ν)

res[1]

9×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.0       0.0
 0.279468  0.349336
 0.403874  0.504843
 0.481627  0.602034
 0.460683  0.575853
 0.457667  0.572084
 0.286507  0.358134
 0.427008  0.53376
 0.465071  0.581338

[:, :, 2] =
 0.0       0.0
 0.279468  0.349336
 0.403874  0.504843
 0.481627  0.602034
 0.460683  0.575853
 0.457667  0.572084
 0.286507  0.358134
 0.427008  0.53376
 0.465071  0.581338

In [183]:
res[2]

9×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.0          0.0
 0.00280969  -0.000237891
 0.00556446  -0.00054442
 0.00724201  -0.00219749
 0.00847471  -0.00440661
 0.0107908   -0.00526147
 0.0125456   -0.00681805
 0.015307    -0.00711628
 0.0          0.0

[:, :, 2] =
  0.0          0.0
  0.000619374  0.00452422
  0.00112893   0.00891116
 -0.000515986  0.010605
 -0.00305058   0.0111868
 -0.00341835   0.0144771
 -0.00490887   0.0163639
 -0.00438604   0.0207674
  0.0          0.0

In [166]:
X = [0.9 0.75 0.9 0.75;
0.6 0.55 0.6 0.55;
0.4 0.78 0.4 0.78;
0.24 1.0 0.24 1.0;
0.26 0.81 0.26 0.81;
0.49 0.61 0.49 0.61;
0.56 0.01 0.56 0.01;
0.4 -0.38 0.4 -0.38]


8×4 Matrix{Float64}:
 0.9    0.75  0.9    0.75
 0.6    0.55  0.6    0.55
 0.4    0.78  0.4    0.78
 0.24   1.0   0.24   1.0
 0.26   0.81  0.26   0.81
 0.49   0.61  0.49   0.61
 0.56   0.01  0.56   0.01
 0.4   -0.38  0.4   -0.38